# Notebook to plot persistence from a dataframe produced by Pers_in_visit and compute the persistence average over many ramps


In [ ]:
from astropy.io import fits
import glob, os, shutil, pickle, bz2, gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import sigmaclip
from scipy.optimize import curve_fit
from scipy.special import gammaincc, gamma
from astropy.wcs import WCS
from astropy.stats import histogram
from itertools import product
from multiprocessing import Pool
from IPython.display import display

%matplotlib notebook

In [ ]:
pwd

In [ ]:
#scsky = 2.75
#skyrad_i = 6
#skyrad_o = 15
#suff = '_sig'+'{:05.2f}'.format(scsky) + '_ri' + '{:05.2f}'.format(skyrad_i) + '_ro' + '{:05.2f}'.format(skyrad_o)

suff = '_dark_only'

In [ ]:
# The project dir 
pdir = '/user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence/'

#The mosaic dir
mdir = pdir+'/Mosaic_hi_res_folder/'

#The dir to save/load the Persistence curves dataframes
sdir = pdir+'/PD_dataframes_dir/'


In [ ]:
#Single and double exponential models to be fitted to the data

def decay1(t,a1,t1):
    e1 = a1*np.exp(-t/t1)
    return e1

def intdec1(t,a1,t1):
    tu = t[1:]
    td = t[:-1]
    k  = -a1*t1
    return k*(np.exp(-tu/t1)-np.exp(-td/t1))/(tu-td)
    
def decay2(t,a1,t1,a2,t2):
    e1 = a1*np.exp(-t/t1)
    e2 = a2*np.exp(-t/t2)
    return e1+e2

def intdec2(t,a1,t1,a2,t2):
    tu = t[1:]
    td = t[:-1]
    k1,k2  = -a1*t1, - a2*t2
    
    return k1*(np.exp(-tu/t1)-np.exp(-td/t1))/(tu-td) + k2*(np.exp(-tu/t2)-np.exp(-td/t2))/(tu-td)

#Single exponential models plus a constant

def intdec1_plusconst(t,a1,t1,q):
    tu = t[1:]
    td = t[:-1]
    k  = -a1*t1
    return k*(np.exp(-tu/t1)-np.exp(-td/t1))/(tu-td) +q

def dec1_plusconst(t,a1,t1,q):
    e1 = a1*np.exp(-t/t1)
    return e1+q


#Shifted power law model

def shpwl(t,t0,A,index):
    return A * ((t+t0)/1000)**index

def intshpwl(t,t0,A,index):
    tu = t[1:]
    td = t[:-1]

    if (index == -1.):
        return A*np.log( (tu+t0)/(td+t0) )
    else:
        return A/(1+index) * ( ((tu+t0)/1000)**(1+index) - ((td+t0)/1000)**(1+index) )/(tu-td)
    
    
#Schechter like model

def schechter(t,phi,alpha,tstar):
    x = t/tstar
    return phi*(x**alpha)*np.exp(-x)

def intschechter(t,phi,alpha,tstar):
    x = t/tstar

    tu = x[1:]
    td = x[:-1]

    g1 = gammaincc(alpha+1,td)
    g2 = gammaincc(alpha+1,tu)
    
    diff = gamma(alpha+1)*(g1-g2)
    
    return phi*diff


#Geometric median calculation function

from scipy.spatial.distance import cdist, euclidean

def geometric_median(X, eps=1e-5):
    y = np.mean(X, 0)

    while True:
        D = cdist(X, [y])
        nonzeros = (D != 0)[:, 0]

        Dinv = 1 / D[nonzeros]
        Dinvs = np.sum(Dinv)
        W = Dinv / Dinvs
        T = np.sum(W * X[nonzeros], 0)

        num_zeros = len(X) - np.sum(nonzeros)
        if num_zeros == 0:
            y1 = T
        elif num_zeros == len(X):
            return y
        else:
            R = (T - y) * Dinvs
            r = np.linalg.norm(R)
            rinv = 0 if r == 0 else num_zeros/r
            y1 = max(0, 1-rinv)*T + min(1, rinv)*y

        if euclidean(y, y1) < eps:
            return y1, D

        y = y1



In [ ]:
#Cell to restore the hf5 store with the dataframe for each visit
#df = pd.DataFrame()

df_v   = pd.DataFrame()
df_l   = pd.DataFrame()
store  = pd.HDFStore(sdir+'DF'+suff+'.h5')
skeys  = store.keys()

print(skeys)

lengths_v = []
lengths_l = []

for i in range(len(skeys)//2):
    df_vh = store.get(skeys[i*2+1])
    df_lh = store.get(skeys[i*2])
    
    lengths_v.append(len(df_vh))
    lengths_l.append(len(df_lh))
    
    print('Number of entries for Visit ',i+1,':',len(df_vh))
    print('Number of unique ramps for Visit ',i+1,':',len(df_lh))

    df_vh['Visit'] = i+1
    df_v = df_v.append(df_vh,ignore_index=True)

    df_lh['Visit'] = i+1
    df_l = df_l.append(df_lh,ignore_index=False)


In [ ]:
#Add an offest to the unique multiindexes, to account for the fact that the uniq multiindex is unqie only
# at the single visit level

for i in range(len(skeys)//2):
    BM_v = df_v['Visit'] ==  i+1
    BM_l = df_l['Visit'] ==  i+1
    offset = np.sum(np.asarray(lengths_l)[:i])
    print(offset)
    df_v.loc[BM_v,'Uniq_multiindex'] += offset
    df_l.loc[BM_l,'Uniq_multiindex'] += offset
    

In [ ]:
print('Total number of entries',len(df_v))
for i in range(len(skeys)//2):
    BM_v = df_v['Visit'] ==  i+1
    display(df_v[BM_v].head())
    


In [ ]:
print('Total number of unique ramps',len(df_l))
for i in range(len(skeys)//2):
    BM_l = df_l['Visit'] ==  i+1
    display(df_l[BM_l].head())


In [ ]:
Stim    = df_l.index.get_level_values('Stim').values
Visit   = df_l['Visit'].values
DQ_stim = df_l.index.get_level_values('DQ_stim').values
Ind_stim    = df_l.index.get_level_values('Ind_stim').values


In [ ]:
fig = plt.figure()

BMplot = DQ_stim == 0

mybins = np.arange(4.6,7.01,0.2)

n,b,p = plt.hist(np.log10(Stim[BMplot]),alpha=0.25,bins=mybins)

for vv in [1,2,3]:
    plt.hist(np.log10(Stim[(Visit == vv) & BMplot]),alpha=0.25,bins=b)
plt.xlabel('Log10(Fluence)')


In [ ]:
#The stimulus levels probed

print('Minimum stimulus:',np.log10(np.min(Stim[DQ_stim == 0])))
print('Maximum stimulus:',np.log10(np.max(Stim[DQ_stim == 0])))


In [ ]:
BM_rng2    = df_v['deltat'] < 3  
BM_rng12   = (df_v['deltat'] < 13) & (df_v['deltat'] > 12) 
BM_rng25   = (df_v['deltat'] < 26) & (df_v['deltat'] > 24) 
BM_rng50   = (df_v['deltat'] < 51) & (df_v['deltat'] > 49) 
BM_rng100  = (df_v['deltat'] < 101) & (df_v['deltat'] > 99) 

BM_pext = df_v['Pers_type'] == 'EXT'
BM_V1   = df_v['Visit'] == 1
BM_V2   = df_v['Visit'] == 2
BM_V3   = df_v['Visit'] == 3

BM_DRK_SPARS25 = (df_v['Pers_type'] != 'EXT') & (df_v['NSAMP'] == 16)

g_vvlu = DQ_stim == 0
g_umix = df_l['Uniq_multiindex'][g_vvlu].values
BM_DQ  = df_v['Uniq_multiindex'].isin(g_umix)

g_vvlu = Ind_stim == 0
g_umix = df_l['Uniq_multiindex'][g_vvlu].values
BM_S0  = df_v['Uniq_multiindex'].isin(g_umix)

g_vvlu = Ind_stim == 1
g_umix = df_l['Uniq_multiindex'][g_vvlu].values
BM_S1  = df_v['Uniq_multiindex'].isin(g_umix)

BM_next = np.zeros(len(df_v),dtype=np.bool_)
for i in range(7):
    g_vvlu = Ind_stim == i
    if np.sum(g_vvlu) > 0 :
        g_umix = df_l['Uniq_multiindex'][g_vvlu].values    
        BMh = (df_v['Uniq_multiindex'].isin(g_umix)) & (df_v['Ind_pers'] == i+1) & (df_v['Pers_type'] == 'EXT')
        BM_next = BM_next | BMh
    


In [ ]:
#levels = [5.0,5.1,5.5,5.75,6.,6.25,6.5,6.75,7.,7.25,7.5,7.75]
#levels = [6.75,7.,7.25]
levels = list(mybins)

mean_res = {'levels': levels, 
            'med_times':[],
            'med_pers':[],
            'Npoints':[],
            'Readme': [#'ALL',
                       #'ALL 25',
                       #'ALL PEXT',
                       #'ALL PDRK',                       
                       #'ALL 25_PEXT',
                       #'ALL 25_PDRK',
                       #'ALL 100',
                       #'ALL 100_PEXT',
                       #'ALL 100_PDRK',
                       #'ALL 2',
                       #'ALL 2_PEXT',
                       #'ALL 2_PDRK',
                       #'ALL 12',
                       #'ALL 12_PEXT',
                       #'ALL 12_PDRK',
                       #'ALL 50',
                       #'ALL 50_PEXT',
                       #'ALL 50_PDRK',                      
                       #'VIS1',
                       #'VIS1 25',
                       #'VIS1 PEXT',
                       #'VIS1 PDRK',                       
                       #'VIS1 25_PEXT',
                       #'VIS1 25_PDRK',
                       #'VIS1 100',
                       #'VIS1 100_PEXT',
                       #'VIS1 100_PDRK',
                       #'VIS1 2',
                       #'VIS1 2_PEXT',
                       #'VIS1 2_PDRK',
                       #'VIS1 12',
                       #'VIS1 12_PEXT',
                       #'VIS1 12_PDRK',
                       #'VIS1 50',
                       #'VIS1 50_PEXT',
                       #'VIS1 50_PDRK',
                       #'VIS2',
                       #'VIS2 25',
                       #'VIS2 PEXT',
                       #'VIS2 PDRK',                       
                       #'VIS2 25_PEXT',
                       #'VIS2 25_PDRK',
                       #'VIS2 100',
                       #'VIS2 100_PEXT',
                       #'VIS2 100_PDRK',
                       #'VIS2 2',
                       #'VIS2 2_PEXT',
                       #'VIS2 2_PDRK',
                       #'VIS2 12',
                       #'VIS2 12_PEXT',
                       #'VIS2 12_PDRK',
                       #'VIS2 50',
                       #'VIS2 50_PEXT',
                       #'VIS2 50_PDRK',
                       #'VIS3',
                       #'VIS3 25',
                       #'VIS3 PEXT',
                       #'VIS3 PDRK',                       
                       #'VIS3 25_PEXT',
                       #'VIS3 25_PDRK',
                       #'VIS3 100',
                       #'VIS3 100_PEXT',
                       #'VIS3 100_PDRK',
                       #'VIS3 2',
                       #'VIS3 2_PEXT',
                       #'VIS3 2_PDRK',
                       #'VIS3 12',
                       #'VIS3 12_PEXT',
                       #'VIS3 12_PDRK',
                       #'VIS3 50',
                       #'VIS3 50_PEXT',
                       #'VIS3 50_PDRK',  
                       
                       #'ALL 25_PDRK_S0',
                       #'ALL 100_PDRK_S0',
                       #'ALL 2_PDRK_S0',
                       #'ALL 12_PDRK_S0',
                       #'ALL 50_PDRK_S0',                      
                       #'VIS1 25_PDRK_S0',
                       #'VIS1 100_PDRK_S0',
                       #'VIS1 2_PDRK_S0',
                       #'VIS1 12_PDRK_S0',
                       #'VIS1 50_PDRK_S0',                      
                       #'VIS2 25_PDRK_S0',
                       #'VIS2 100_PDRK_S0',
                       #'VIS2 2_PDRK_S0',
                       #'VIS2 12_PDRK_S0',
                       #'VIS2 50_PDRK_S0',                      
                       #'VIS3 25_PDRK_S0',
                       #'VIS3 100_PDRK_S0',
                       #'VIS3 2_PDRK_S0',
                       #'VIS3 12_PDRK_S0',
                       #'VIS3 50_PDRK_S0',
                       
                       'ALL 25_PDRK_S1',
                       'ALL 100_PDRK_S1',
                       'ALL 2_PDRK_S1',
                       'ALL 12_PDRK_S1',
                       'ALL 50_PDRK_S1',                      
                       'VIS1 25_PDRK_S1',
                       'VIS1 100_PDRK_S1',
                       'VIS1 2_PDRK_S1',
                       'VIS1 12_PDRK_S1',
                       'VIS1 50_PDRK_S1',                      
                       'VIS2 25_PDRK_S1',
                       'VIS2 100_PDRK_S1',
                       'VIS2 2_PDRK_S1',
                       'VIS2 12_PDRK_S1',
                       'VIS2 50_PDRK_S1',                      
                       'VIS3 25_PDRK_S1',
                       'VIS3 100_PDRK_S1',
                       'VIS3 2_PDRK_S1',
                       'VIS3 12_PDRK_S1',
                       'VIS3 50_PDRK_S1'                      
                      ]
            }    


step = 5.
    
for lev_dw,lev_up in zip(mean_res['levels'][0:-1],mean_res['levels'][1:]):
    
    print(lev_dw,lev_up)
    
    g_vvlu = ( np.log10(Stim) >= lev_dw ) & ( np.log10(Stim) < lev_up )
    g_umix = df_l['Uniq_multiindex'][g_vvlu].values
    BM_lv  = df_v['Uniq_multiindex'].isin(g_umix)

    gBMs = []
    
    BM_lvDQ = BM_lv & BM_DQ
      
#    gBMs.append(BM_lvDQ)    
#    gBMs.append(BM_lvDQ & BM_rng25)    
#    gBMs.append(BM_lvDQ & BM_pext)    
#    gBMs.append(BM_lvDQ & (~BM_pext))    
#    gBMs.append(BM_lvDQ & BM_rng25 & BM_pext)    
#    gBMs.append(BM_lvDQ & BM_rng25 & (~BM_pext))    
#    gBMs.append(BM_lvDQ & BM_rng100)    
#    gBMs.append(BM_lvDQ & BM_rng100 & BM_pext)    
#    gBMs.append(BM_lvDQ & BM_rng100 & (~BM_pext))    
#    gBMs.append(BM_lvDQ & BM_rng2)    
#    gBMs.append(BM_lvDQ & BM_rng2 & BM_pext)    
#    gBMs.append(BM_lvDQ & BM_rng2 & (~BM_pext))    
#    gBMs.append(BM_lvDQ & BM_rng12)    
#    gBMs.append(BM_lvDQ & BM_rng12 & BM_pext)    
#    gBMs.append(BM_lvDQ & BM_rng12 & (~BM_pext))    
#    gBMs.append(BM_lvDQ & BM_rng50)    
#    gBMs.append(BM_lvDQ & BM_rng50 & BM_pext)    
#    gBMs.append(BM_lvDQ & BM_rng50 & (~BM_pext))    

#    gBMs.append(BM_V1 & BM_lvDQ)    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng25)    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_pext)    
#    gBMs.append(BM_V1 & BM_lvDQ & (~BM_pext))    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng25 & BM_pext)    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng25 & (~BM_pext))    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng100)    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng100 & BM_pext)    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng100 & (~BM_pext))    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng2)    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng2 & BM_pext)    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng2 & (~BM_pext))    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng12)    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng12 & BM_pext)    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng12 & (~BM_pext))    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng50)    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng50 & BM_pext)    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng50 & (~BM_pext))    

#    gBMs.append(BM_V2 & BM_lvDQ)    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng25)    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_pext)    
#    gBMs.append(BM_V2 & BM_lvDQ & (~BM_pext))    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng25 & BM_pext)    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng25 & (~BM_pext))    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng100)    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng100 & BM_pext)    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng100 & (~BM_pext))    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng2)    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng2 & BM_pext)    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng2 & (~BM_pext))    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng12)    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng12 & BM_pext)    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng12 & (~BM_pext))    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng50)    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng50 & BM_pext)    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng50 & (~BM_pext))    

#    gBMs.append(BM_V3 & BM_lvDQ)    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng25)    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_pext)    
#    gBMs.append(BM_V3 & BM_lvDQ & (~BM_pext))    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng25 & BM_pext)    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng25 & (~BM_pext))    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng100)    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng100 & BM_pext)    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng100 & (~BM_pext))    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng2)    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng2 & BM_pext)    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng2 & (~BM_pext))    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng12)    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng12 & BM_pext)    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng12 & (~BM_pext))    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng50)    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng50 & BM_pext)    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng50 & (~BM_pext))    

#    gBMs.append(BM_lvDQ & BM_rng25 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_lvDQ & BM_rng100 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_lvDQ & BM_rng2 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_lvDQ & BM_rng12 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_lvDQ & BM_rng50 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng25 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng100 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng2 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng12 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_V1 & BM_lvDQ & BM_rng50 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng25 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng100 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng2 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng12 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_V2 & BM_lvDQ & BM_rng50 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng25 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng100 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng2 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng12 & (~BM_pext) & BM_S0)    
#    gBMs.append(BM_V3 & BM_lvDQ & BM_rng50 & (~BM_pext) & BM_S0)    

    gBMs.append(BM_lvDQ & BM_rng25 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_lvDQ & BM_rng100 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_lvDQ & BM_rng2 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_lvDQ & BM_rng12 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_lvDQ & BM_rng50 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_V1 & BM_lvDQ & BM_rng25 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_V1 & BM_lvDQ & BM_rng100 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_V1 & BM_lvDQ & BM_rng2 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_V1 & BM_lvDQ & BM_rng12 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_V1 & BM_lvDQ & BM_rng50 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_V2 & BM_lvDQ & BM_rng25 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_V2 & BM_lvDQ & BM_rng100 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_V2 & BM_lvDQ & BM_rng2 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_V2 & BM_lvDQ & BM_rng12 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_V2 & BM_lvDQ & BM_rng50 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_V3 & BM_lvDQ & BM_rng25 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_V3 & BM_lvDQ & BM_rng100 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_V3 & BM_lvDQ & BM_rng2 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_V3 & BM_lvDQ & BM_rng12 & (~BM_pext) & BM_S1)    
    gBMs.append(BM_V3 & BM_lvDQ & BM_rng50 & (~BM_pext) & BM_S1)    

    
    mth = []
    mph = []
    nph = []
    
    for i,gBM in enumerate(gBMs):
        print(mean_res['Readme'][i],np.sum(gBM),np.sum(BM_lv))
        
        tmed   = []
        medsig = []
        npts   = []
        
        if (np.sum(gBM) > 0 ):
            df2 = df_v[gBM]
            tmin = 0.
            times = df2['tfromstim'].values-0.5*df2['deltat'].values
            tmax = np.max(times)

            while tmin <= tmax:
                BM   = ( times > (tmin-step/2.)) & ( times <= (tmin+ step/2.))
                tmin = tmin + step
                nn   = np.sum(BM)
                if (nn > 5):
                    y = df2['meancurr'][BM].values
                    medsig.append(np.nanmedian(sigmaclip(y[np.isfinite(y)],2.75,2.75)[0]))
                    x = times[BM]
                    tmed.append(np.median(x))
                    npts.append(nn)
    
        mth.append(tmed)
        mph.append(medsig)
        nph.append(npts)
        
    mean_res['med_times'].append(mth)
    mean_res['med_pers'].append(mph)
    mean_res['Npoints'].append(nph)


with open(sdir+'/Mean_res'+suff+'.pickle', 'wb') as f:
    pickle.dump(mean_res, f, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
#scsky = 2.10
#skyrad_i = 4
#skyrad_o = 13

#suff = '_sig'+'{:05.2f}'.format(scsky) + '_ri' + '{:05.2f}'.format(skyrad_i) + '_ro' + '{:05.2f}'.format(skyrad_o)

#with open(sdir+'/Mean_res'+suff+'.pickle', 'rb') as f:
#    mean_res = pickle.load(f)

with open(sdir+'/Mean_res.pickle', 'rb') as f:
    mean_res_drk = pickle.load(f)

    


In [ ]:
#fig = plt.figure(figsize=(18,3*(len(mean_res['levels'])-1)))

#name_to_plot = ['VIS1 25_PEXT_P3','VIS1 25_PEXT_P4','VIS1 25_PEXT_P5','VIS1 25_PEXT_P6','VIS1 25_PDRK_P7','VIS1 25_PDRK_P8','VIS1 25_PDRK_P9']
#name_to_plot = ['ALL 25_PEXT','ALL 25_PDRK','VIS1 25_PEXT','VIS2 25_PEXT','VIS3 25_PEXT','VIS1 25_NEXT','VIS1 25_NEXT_OR_PDRK']
name_to_plot = ['ALL 25_PEXT','ALL 25_PDRK']

#name_to_plot = ['ALL']
ind_to_plot = [mean_res['Readme'].index(name) for name in name_to_plot]

ncols = 4
nrows =  (len(mean_res['levels'])-1) // ncols

if ((len(mean_res['levels'])-1) % ncols) > 0:
    nrows = nrows+1

ax = []

plt.style.use('bmh')
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 13
plt.rcParams['axes.labelsize'] = 13
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['xtick.labelsize'] = 11
plt.rcParams['ytick.labelsize'] = 11
plt.rcParams['legend.fontsize'] = 9
plt.rcParams['figure.titlesize'] = 13

#col_list = ['#909AEE','#8B4500','#008B45','#CD9B1D','#99BB00','#9400D3','#FF00FF','#00688B','#FF3030',
#            '#1A9ABF','#B8000E','#B45008','#B1DCD9','#B0099B','#0D3940','#0FFFF0','#88B006','#030FF3',
#            '#2A9AFB','#118753','#B45008','#B1DCD9','#B0099B','#0D3940','#0FFFF0','#88B006','#030FF3',
#            '#9E11BF','#B8000E','#B45008','#B1DCD9','#B0099B','#0D3940','#0FFFF0','#88B006','#030FF3',
#            '#BEDCAC','#B8000E','#B45008','#B1DCD9','#B0099B','#0D3940','#0FFFF0','#88B006','#030FF3']


fig, ax = plt.subplots(nrows, ncols, sharex=True, sharey=True, figsize=(5*ncols,3.5*nrows))

for i,(lev_dw,lev_up) in enumerate(zip(mean_res['levels'][0:-1],mean_res['levels'][1:])):
   
    k = i%ncols
    l = i//ncols
       
    for mm,j in enumerate(ind_to_plot):
        if 'PEXT' in name_to_plot[mm]:
            mrk = 'o'
            sz=12
        else:
            mrk = '*'
            sz=30
                        
        ax[l,k].scatter(mean_res['med_times'][i][j],mean_res['med_pers'][i][j],#color=col_list[j],
                     label='N: '+'{:3.1e}'.format(1.*np.sum(mean_res['Npoints'][i][j]))+' ; '+mean_res['Readme'][j],s=sz, marker=mrk)
        
        
        
    ax[l,k].set_xscale('log')
    ax[l,k].set_yscale('log')
    meanlev = 0.5*(mean_res['levels'][i]+mean_res['levels'][i+1])
    titlestring = 'Log$_{10}$ (Stim.[e$^{-}$]): ' + '{:5.2f}'.format(meanlev) + '; ' + '{:6.2f}'.format(10**meanlev/80000.) +' x saturation' 
    ax[l,k].text(0.25,0.875,titlestring,verticalalignment='bottom', horizontalalignment='left',transform=ax[l,k].transAxes,bbox={'facecolor':'gray', 'alpha':.1, 'pad':1},fontsize=11)

    ax[l,k].legend(fontsize=9,loc=3)
    
    ax[l,k].set_ylim(0.009,25)
    ax[l,k].set_xlim(40,10000)
    
    ax[l,k].plot(np.array([1,10000]),700*np.array([1,1/10000.]),'--',c='black')

    if l==(nrows-1):
        ax[l,k].set_xlabel('Time [s]',fontsize=15)
    if k==0:
        ax[l,k].set_ylabel('Mean current [e$^{-}$/s]',fontsize=15)
    ax[l,k].set_axis_bgcolor('#FFFFFF')

    
name_to_plot = ['ALL 25_PDRK']

ind_to_plot = [mean_res_drk['Readme'].index(name) for name in name_to_plot]
    
for i,(lev_dw,lev_up) in enumerate(zip(mean_res_drk['levels'][0:-1],mean_res_drk['levels'][1:])):
   
    k = i%ncols
    l = i//ncols
       
    for mm,j in enumerate(ind_to_plot):
        if 'PEXT' in name_to_plot[mm]:
            mrk = 'o'
            sz=12
        else:
            mrk = '*'
            sz=30
                        
        ax[l,k].scatter(mean_res_drk['med_times'][i][j],mean_res_drk['med_pers'][i][j],#color=col_list[j],
                     label='N: '+'{:3.1e}'.format(1.*np.sum(mean_res_drk['Npoints'][i][j]))+' ; '+mean_res_drk['Readme'][j],s=sz, marker=mrk)
        
        
    

    

plt.tight_layout(pad=.2)

